In [1]:
from pathlib import Path
from time import time
import pandas as pd
from videosentimentanalysis.usecases.download_video_use_case import VideoUseCase
from videosentimentanalysis.adapters.logging.custom_logger import CustomLogger
from videosentimentanalysis.adapters.logging.loguru_logger import LoguruLogger
from videosentimentanalysis.adapters.extract_audio.moviepy_extraction import MoviePyAudioExtractor
from videosentimentanalysis.usecases.video_utility_use_case import VideoUtilityUseCase
from videosentimentanalysis.adapters.transcribe_audio.speechrecognition import SpeechRecognition
from videosentimentanalysis.adapters.perform_sentiment_analysis.textblob_analysis import TextBlobAnalysis
from videosentimentanalysis.adapters.translate_text.google_translate import TextTranslation
from videosentimentanalysis.usecases.protocols.translate_text import LanguageOptions
from videosentimentanalysis.adapters.extract_emotions.text2emotion_extractor import GetEmotions
from videosentimentanalysis.domain.audio import Audio
import json
from threading import Thread
from multiprocessing import Process, Manager


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Constants

In [2]:
OUTPUT_FOLDER= Path("./output/")


To run this project you need to have the following installed:
- ffmpeg
- python3.12
- poetry

## Setup Adapters
### Logging


In [3]:
custom_logger = CustomLogger(Path("./download_log.txt"))
loguru_logger = LoguruLogger(Path("./loguru_logger.log"))

### Video Utilities

In [4]:

moviepy_extractor = MoviePyAudioExtractor(Path("./tmp_audio"))
speechrecognition = SpeechRecognition()
sentimentanalysis = TextBlobAnalysis()
text_translation = TextTranslation()
emotion_extraction = GetEmotions()

Load all the Youtube URLs to a Pydantic Model called RawURLVideo

In [5]:
video_use_case = VideoUseCase(video_output_directory=Path("tmp_videos"), logger=custom_logger)

In [6]:
raw_url_videos=video_use_case.load_raw_url_video(Path('./video_urls.txt'))
raw_url_videos

[RawVideoUrl(url='https://www.youtube.com/watch?v=G025oxyWv0E\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=G7xFfezsJ1s\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=qAsdnGJFbt8\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=9dpwVc6Olo0\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=PtjKDyLMtio\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=ah_WSxzN8v4\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=qRdzw2Osl8o\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=LkWQvzrv6gI\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=2P1nE57dvNw\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=vvkIF0NlIzA')]

In [7]:
sequential_time_taken = time()
video_use_case.download_raw_videos_sequentially(raw_url_videos)
sequential_time_taken = time() - sequential_time_taken

2024-06-24 11:39:34.385104+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' {} () 
2024-06-24 11:39:35.828131+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/2f0d4664-118d-418e-be31-f2dc0e7db8cd.mp4 {} () 

2024-06-24 11:39:35.955946+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' {} () 
2024-06-24 11:39:37.001015+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/ca8b3efc-bee6-4274-97da-0a53184b89b0.mp4 {} () 

2024-06-24 11:39:37.102477+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' {} () 
2024-06-24 11:39:38.356351+00:00 INFO: Downloaded video from url='https://

In [8]:
threading_time_taken = time()
video_use_case.download_raw_videos_threading(raw_url_videos)
threading_time_taken = time() - threading_time_taken

2024-06-24 11:39:53.702750+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' on thread 2 {} () 

2024-06-24 11:39:53.702192+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=G025oxyWv0E\n' on thread 0 {} () 

2024-06-24 11:39:53.703039+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n' on thread 3 {} () 

2024-06-24 11:39:53.702575+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' on thread 1 {} () 

2024-06-24 11:39:53.703147+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n' on thread 4 {} () 

2024-06-24 11:39:53.806264+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' {} () 

2024-06-24 11:39:53.819448+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' {} () 

2024-06-24 11:39:53.820559+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com

In [9]:
process_time_taken = time()
videos=video_use_case.download_raw_videos_multiprocessing(raw_url_videos)
process_time_taken = time() - process_time_taken

2024-06-24 11:40:00.875318+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=G7xFfezsJ1s\n' on process 5 {} () 

2024-06-24 11:40:00.999609+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=G7xFfezsJ1s\n' {} () 

2024-06-24 11:40:02.320543+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=G7xFfezsJ1s\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/a753e3e6-4bfd-4851-844e-0a7a3f47388f.mp4 {} () 

2024-06-24 11:40:00.849434+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' on process 1 {} () 

2024-06-24 11:40:00.992099+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' {} () 

2024-06-24 11:40:02.432903+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/b83a5b09-81

## Results of each method for downloading videos and time taken

In [10]:
pd.DataFrame({
    "Method": ["Sequential", "Threading", "Multiprocessing"],
    "Time Taken": [sequential_time_taken, threading_time_taken, process_time_taken]
})

,Method,Time Taken
0,Sequential,19.417417
1,Threading,6.947092
2,Multiprocessing,5.106775


The above shows the difference in performance when used downloading videos sequentially, using threading or multiprocessing. Unsurprisingly, the sequential method is the slowest. As this is a CPU bound task, the multiprocessing method is the quickest one.

In [11]:

for video_index in range(len(videos)):
    video=videos[video_index]
    video_utlity_use_case=VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    
    video_output_folder= OUTPUT_FOLDER / video.title
    video_output_folder.mkdir(exist_ok=True, parents=True)
    
    video.move(video_output_folder)
    print(video.local_storage_path)
    
    audio = video_utlity_use_case.extract_audio()
    audio.move(video_output_folder)
    
    with open(video_output_folder / "sentimental_analysis.json", mode="w") as sentimental_analysis_file:
        sentimental_analysis_file.write(json.dumps(video_utlity_use_case.get_sentiment_analysis()))
    
    with open(video_output_folder / "transcribe_text.txt", mode="w") as transcribe_text_file:
        transcribe_text_file.write(video_utlity_use_case.transcribe_audio())
    
    with open(video_output_folder / "translate_text.txt", mode="w") as translate_text_file:
        translate_text_file.write(video_utlity_use_case.translate_text(source_lang=LanguageOptions.ENGLISH, target_lang=LanguageOptions.SPANISH))
    
    with open(video_output_folder / "detect_emotions.json", mode="w") as detect_emotions_file:
        detect_emotions_file.write(json.dumps(video_utlity_use_case.detect_emotions()))
    


output/If Educational Videos Were Filmed Like Music Videos/1307d190-1aa4-4aee-b36e-beb527ff99a8.mp4
2024-06-24 11:40:05.755471+00:00 INFO: Extracting audio from video If Educational Videos Were Filmed Like Music Videos {} () 

MoviePy - Writing audio in tmp_audio/be4478c5-338f-43a5-ad3f-026e3aa603b9.wav


MoviePy - Done.
2024-06-24 11:40:06.361896+00:00 INFO: Returning cached audio for video If Educational Videos Were Filmed Like Music Videos {} () 

2024-06-24 11:40:06.361992+00:00 INFO: Transcribing audio from video If Educational Videos Were Filmed Like Music Videos {} () 
2024-06-24 11:40:17.425783+00:00 INFO: Performing sentiment analysis on video If Educational Videos Were Filmed Like Music Videos {} () 

2024-06-24 11:40:17.446853+00:00 INFO: Returning cached transcribed audio for video If Educational Videos Were Filmed Like Music Videos {} () 

2024-06-24 11:40:17.447044+00:00 INFO: Returning cached transcribed audio for video If Educational Videos Were Filmed Like Music Videos {} () 
2024-06-24 11:40:17.909922+00:00 INFO: Returning cached transcribed audio for video If Educational Videos Were Filmed Like Music Videos {} () 
output/Casually Explained: Guide to College and University/b83a5b09-81e2-48a5-95ce-b64c6ede8bfb.mp4
2024-06-24 11:40:19.389374+00:00 INFO: Extracting audio 

MoviePy - Done.
2024-06-24 11:40:20.196605+00:00 INFO: Returning cached audio for video Casually Explained: Guide to College and University {} () 

2024-06-24 11:40:20.196725+00:00 INFO: Transcribing audio from video Casually Explained: Guide to College and University {} () 
2024-06-24 11:40:35.546287+00:00 INFO: Performing sentiment analysis on video Casually Explained: Guide to College and University {} () 

2024-06-24 11:40:35.551815+00:00 INFO: Returning cached transcribed audio for video Casually Explained: Guide to College and University {} () 

2024-06-24 11:40:35.552038+00:00 INFO: Returning cached transcribed audio for video Casually Explained: Guide to College and University {} () 
2024-06-24 11:40:35.813906+00:00 INFO: Returning cached transcribed audio for video Casually Explained: Guide to College and University {} () 
output/Potter Puppet Pals: The Mysterious Ticking Noise/fd2953d4-b07b-4689-b31f-f7ab45c51e28.mp4
2024-06-24 11:40:37.550255+00:00 INFO: Extracting audio fro

MoviePy - Done.
2024-06-24 11:40:38.128681+00:00 INFO: Returning cached audio for video Potter Puppet Pals: The Mysterious Ticking Noise {} () 

2024-06-24 11:40:38.128775+00:00 INFO: Transcribing audio from video Potter Puppet Pals: The Mysterious Ticking Noise {} () 
2024-06-24 11:40:41.599042+00:00 INFO: Performing sentiment analysis on video Potter Puppet Pals: The Mysterious Ticking Noise {} () 

2024-06-24 11:40:41.600322+00:00 INFO: Returning cached transcribed audio for video Potter Puppet Pals: The Mysterious Ticking Noise {} () 

2024-06-24 11:40:41.600521+00:00 INFO: Returning cached transcribed audio for video Potter Puppet Pals: The Mysterious Ticking Noise {} () 
2024-06-24 11:40:42.797487+00:00 INFO: Returning cached transcribed audio for video Potter Puppet Pals: The Mysterious Ticking Noise {} () 

output/Fastest Rap EVER/2fedeaaa-2987-4b85-bc75-52c6ee476741.mp4
2024-06-24 11:40:42.920374+00:00 INFO: Extracting audio from video Fastest Rap EVER {} () 
MoviePy - Writing

MoviePy - Done.
2024-06-24 11:40:43.341014+00:00 INFO: Returning cached audio for video Fastest Rap EVER {} () 

2024-06-24 11:40:43.341119+00:00 INFO: Transcribing audio from video Fastest Rap EVER {} () 


...

MoviePy - Done.
2024-06-24 11:41:53.793098+00:00 INFO: Returning cached audio for video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} () 

2024-06-24 11:41:53.793216+00:00 INFO: Transcribing audio from video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} () 
2024-06-24 11:42:00.606336+00:00 INFO: Performing sentiment analysis on video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} () 

2024-06-24 11:42:00.608886+00:00 INFO: Returning cached transcribed audio for video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} () 

2024-06-24 11:42:00.609072+00:00 INFO: Returning cached transcribed audio for video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} () 

2024-06-24 11:42:00.793229+00:00 INFO: Returning cached transcribed audio for video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} () 
output/Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim/d403fe05-5818-4535-b56b-9fd

MoviePy - Done.
2024-06-24 11:42:02.026634+00:00 INFO: Returning cached audio for video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} () 

2024-06-24 11:42:02.026762+00:00 INFO: Transcribing audio from video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} () 
2024-06-24 11:42:09.458761+00:00 INFO: Performing sentiment analysis on video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} () 

2024-06-24 11:42:09.461125+00:00 INFO: Returning cached transcribed audio for video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} () 

2024-06-24 11:42:09.461300+00:00 INFO: Returning cached transcribed audio for video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} () 
2024-06-24 11:42:09.724019+00:00 INFO: Returning cached transcribed audio for video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} () 
output/Pirates of the Caribbean - The Curse of the Black Pearl - J

MoviePy - Done.
2024-06-24 11:42:10.843830+00:00 INFO: Returning cached audio for video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} () 

2024-06-24 11:42:10.843934+00:00 INFO: Transcribing audio from video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} () 
2024-06-24 11:42:18.642600+00:00 INFO: Performing sentiment analysis on video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} () 

2024-06-24 11:42:18.643899+00:00 INFO: Returning cached transcribed audio for video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} () 

2024-06-24 11:42:18.644102+00:00 INFO: Returning cached transcribed audio for video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} () 
2024-06-24 11:42:18.890650+00:00 INFO: Returning cached transcribed audio for video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} () 

output/The Kai N' Speed

MoviePy - Done.
2024-06-24 11:42:19.517006+00:00 INFO: Returning cached audio for video The Kai N' Speed Show | Official Trailer {} () 

2024-06-24 11:42:19.517102+00:00 INFO: Transcribing audio from video The Kai N' Speed Show | Official Trailer {} () 
2024-06-24 11:42:32.202708+00:00 INFO: Performing sentiment analysis on video The Kai N' Speed Show | Official Trailer {} () 

2024-06-24 11:42:32.204869+00:00 INFO: Returning cached transcribed audio for video The Kai N' Speed Show | Official Trailer {} () 

2024-06-24 11:42:32.205077+00:00 INFO: Returning cached transcribed audio for video The Kai N' Speed Show | Official Trailer {} () 
2024-06-24 11:42:32.444848+00:00 INFO: Returning cached transcribed audio for video The Kai N' Speed Show | Official Trailer {} () 
output/Futurama - The End of the Universe/4cf36ab3-c138-4bca-a725-5a6931b546dc.mp4
2024-06-24 11:42:32.780834+00:00 INFO: Extracting audio from video Futurama - The End of the Universe {} () 
MoviePy - Writing audio in tmp

MoviePy - Done.
2024-06-24 11:42:33.514143+00:00 INFO: Returning cached audio for video Futurama - The End of the Universe {} () 

2024-06-24 11:42:33.514231+00:00 INFO: Transcribing audio from video Futurama - The End of the Universe {} () 
2024-06-24 11:42:40.273577+00:00 INFO: Performing sentiment analysis on video Futurama - The End of the Universe {} () 

2024-06-24 11:42:40.275903+00:00 INFO: Returning cached transcribed audio for video Futurama - The End of the Universe {} () 

2024-06-24 11:42:40.276095+00:00 INFO: Returning cached transcribed audio for video Futurama - The End of the Universe {} () 
2024-06-24 11:42:40.661009+00:00 INFO: Returning cached transcribed audio for video Futurama - The End of the Universe {} () 


### Sequential time for extract audio

In [12]:
videos=video_use_case.download_raw_videos_multiprocessing(raw_url_videos)

2024-06-24 11:42:41.437885+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n' on process 3 {} () 

2024-06-24 11:42:41.566238+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n' {} () 

2024-06-24 11:42:42.709402+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/84f4bd80-0938-4d3b-b25b-fffd33e90b90.mp4 {} () 

2024-06-24 11:42:41.468476+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n' on process 4 {} () 

2024-06-24 11:42:41.597424+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n' {} () 

2024-06-24 11:42:42.922784+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/a72b5316-c1

In [13]:
extract_audio_time_seq = time()
for video in videos:
    video_utlity_use_case=VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    video_utlity_use_case.extract_audio()

extract_audio_time_seq = time() -extract_audio_time_seq
print(f"Time taken for sequential extraction is {extract_audio_time_seq} seconds")

2024-06-24 11:42:46.177032+00:00 INFO: Extracting audio from video If Educational Videos Were Filmed Like Music Videos {} () 
MoviePy - Writing audio in tmp_audio/574dbbb2-fe89-4b93-aaa5-736d318bf62a.wav


MoviePy - Done.
2024-06-24 11:42:46.913261+00:00 INFO: Extracting audio from video Casually Explained: Guide to College and University {} () 
MoviePy - Writing audio in tmp_audio/14fcd604-10cb-4785-b4d6-30ff2d0017a6.wav


MoviePy - Done.
2024-06-24 11:42:47.758125+00:00 INFO: Extracting audio from video Potter Puppet Pals: The Mysterious Ticking Noise {} () 
MoviePy - Writing audio in tmp_audio/2317814a-6f2f-4e9a-a927-d3bde30ae322.wav


MoviePy - Done.
2024-06-24 11:42:48.395002+00:00 INFO: Extracting audio from video Fastest Rap EVER {} () 
MoviePy - Writing audio in tmp_audio/5d798ecf-83f2-4d27-8dcb-1d672b56a7aa.wav


MoviePy - Done.
2024-06-24 11:42:48.875719+00:00 INFO: Extracting audio from video THE PARTS OF A CELL SONG | Science Music Video {} () 
MoviePy - Writing audio in tmp_audio/8baa53d7-a98f-42ce-8f7b-f9ac8adbba64.wav


...

MoviePy - Done.
2024-06-24 11:42:52.894066+00:00 INFO: Extracting audio from video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} () 
MoviePy - Writing audio in tmp_audio/daea2460-60c0-42be-8823-d82722a29647.wav


MoviePy - Done.
2024-06-24 11:42:53.503692+00:00 INFO: Extracting audio from video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} () 
MoviePy - Writing audio in tmp_audio/251a8530-a205-43d2-aed5-9af96473c54d.wav


MoviePy - Done.
2024-06-24 11:42:54.066777+00:00 INFO: Extracting audio from video The Kai N' Speed Show | Official Trailer {} () 
MoviePy - Writing audio in tmp_audio/5b5c2b43-b397-4346-be95-7afa8eab0beb.wav


MoviePy - Done.
2024-06-24 11:42:54.587759+00:00 INFO: Extracting audio from video Futurama - The End of the Universe {} () 
MoviePy - Writing audio in tmp_audio/1e0d44f9-ac6b-4ebf-a4d7-f962aee83bbd.wav


MoviePy - Done.
Time taken for sequential extraction is 9.156676054000854 seconds


### Threading time for extract audio

In [14]:
def threading_wrapper_extractor(results: list[Audio],index: int,video_utlity_use_case :  VideoUtilityUseCase):
    custom_logger.log(f"Processing {video_utlity_use_case.video.title} on thread {index}")
    results[index]=video_utlity_use_case.extract_audio()

In [15]:
extract_audio_time_threading = time()
results = [None] * len(videos)
threads = []
for index, video in enumerate(videos):
    video_utlity_use_case= VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    thread = Thread(target=threading_wrapper_extractor, args=(results,index,video_utlity_use_case))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
extract_audio_time_threading = time() -extract_audio_time_threading
print(f"Time taken for threading extraction is {extract_audio_time_threading} seconds")

2024-06-24 11:42:55.340718+00:00 INFO: Processing If Educational Videos Were Filmed Like Music Videos on thread 0 {} () 

2024-06-24 11:42:55.341265+00:00 INFO: Extracting audio from video If Educational Videos Were Filmed Like Music Videos {} () 

2024-06-24 11:42:55.341303+00:00 INFO: Processing Potter Puppet Pals: The Mysterious Ticking Noise on thread 2 {} () 

2024-06-24 11:42:55.340960+00:00 INFO: Processing Casually Explained: Guide to College and University on thread 1 {} () 

2024-06-24 11:42:55.341457+00:00 INFO: Processing Fastest Rap EVER on thread 3 {} () 

2024-06-24 11:42:55.341580+00:00 INFO: Extracting audio from video Potter Puppet Pals: The Mysterious Ticking Noise {} () 

2024-06-24 11:42:55.341717+00:00 INFO: Processing THE PARTS OF A CELL SONG | Science Music Video on thread 4 {} () 

2024-06-24 11:42:55.341599+00:00 INFO: Extracting audio from video Casually Explained: Guide to College and University {} () 

2024-06-24 11:42:55.370759+00:00 INFO: Extracting audio

chunk:   0%|          | 2/2798 [00:00<00:48, 58.13it/s, now=None]
                                                                 
chunk:   0%|          | 7/2798 [00:00<00:29, 93.07it/s, now=None]
                                                                 e]
chunk:   0%|          | 10/2798 [00:00<00:29, 93.18it/s, now=None] 
                                                                  ]
chunk:   0%|          | 10/2798 [00:00<00:29, 93.18it/s, now=None] 
                                                                  ]
chunk:   0%|          | 10/2798 [00:00<00:29, 93.18it/s, now=None] 
                                                                  ]
chunk:   0%|          | 18/4068 [00:00<00:23, 172.64it/s, now=None]

MoviePy - Writing audio in tmp_audio/96f509d5-41f0-4706-87a6-40e64ce50ff8.wav
MoviePy - Writing audio in tmp_audio/53c7455f-7af6-4a15-8d83-bef218ab5281.wav
MoviePy - Writing audio in tmp_audio/5e69bd00-d79b-4358-b6a7-58a3250a6b39.wav
MoviePy - Writing audio in tmp_audio/d401194d-9a89-4f19-8b8f-711fa7422d66.wav
MoviePy - Writing audio in tmp_audio/305fc584-c0d8-477f-b600-19d1569f513b.wav
MoviePy - Writing audio in tmp_audio/5ce584e7-e881-4ce0-872a-8637c804546d.wav


                                                                  
chunk:   0%|          | 10/2798 [00:00<00:29, 93.18it/s, now=None] 
                                                                  ]
chunk:   0%|          | 10/2798 [00:00<00:29, 93.18it/s, now=None] 
                                                                  ]
chunk:   0%|          | 10/2798 [00:00<00:29, 93.18it/s, now=None] 
                                                                  ]
chunk:   0%|          | 10/2798 [00:00<00:29, 93.18it/s, now=None] 
                                                                  ]
chunk:   0%|          | 10/2798 [00:00<00:29, 93.18it/s, now=None] 
                                                                  ]
chunk:   0%|          | 10/2798 [00:00<00:29, 93.18it/s, now=None]]


MoviePy - Writing audio in tmp_audio/e8fbeb97-75e6-4fff-b683-972e9cd1506b.wav
MoviePy - Writing audio in tmp_audio/2e40c2cd-ca1c-43de-b871-520e9fd38c95.wav
MoviePy - Writing audio in tmp_audio/faa7fe1b-9b39-4798-8fe7-039e7a01d7c8.wav
MoviePy - Writing audio in tmp_audio/8f5a10e3-7cc5-4002-8d2c-6abda151ba78.wav
MoviePy - Writing audio in tmp_audio/91377f4a-d319-41cf-a88c-6ac9ef236fce.wav
MoviePy - Writing audio in tmp_audio/11448df1-90ba-4da8-a85b-f720986b344c.wav


...

MoviePy - Done.
MoviePy - Done.
MoviePy - Done.


chunk:  70%|███████   | 3718/5303 [00:08<00:01, 853.81it/s, now=None]






chunk:  80%|███████▉  | 3685/4635 [00:08<00:01, 822.02it/s, now=None]






chunk:  80%|███████▉  | 3687/4635 [00:08<00:01, 832.77it/s, now=None]


chunk:  95%|█████████▌| 3878/4068 [00:08<00:00, 950.20it/s, now=None]

chunk:  72%|███████▏  | 3825/5303 [00:08<00:01, 990.15it/s, now=None]






chunk:  83%|████████▎ | 3858/4635 [00:08<00:00, 1093.91it/s, now=None]


chunk: 100%|█████████▉| 4062/4068 [00:08<00:00, 1207.26it/s, now=None]

chunk:  76%|███████▌  | 4007/5303 [00:08<00:01, 1232.92it/s, now=None]
                                                                      


                                                             

                                                                






                                                 


chunk:  83%|████████▎ | 4027/4834 [00:08<00:00, 1227.40it/s, now=None]

chunk:  76%|███████▌  | 4025/5303 [00:08<00:01, 1232.92it/s, now=None]






chunk

MoviePy - Done.









chunk:  99%|█████████▉| 4607/4635 [00:08<00:00, 2094.48it/s, now=None]






                                                                      


                                                             

                                                                


chunk:  98%|█████████▊| 4736/4834 [00:08<00:00, 2034.39it/s, now=None]

chunk:  88%|████████▊ | 4693/5303 [00:08<00:00, 1971.17it/s, now=None]


                                                                      

                                                                

chunk:  91%|█████████ | 4807/5303 [00:08<00:00, 1971.17it/s, now=None]

chunk:  94%|█████████▍| 4982/5303 [00:08<00:00, 2701.25it/s, now=None]

                                                                      

MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
Time taken for threading extraction is 10.704738140106201 seconds


### Processing time for extract audio

In [16]:
extract_audio_time_processes = time()
manager = Manager()
results = manager.list([None] * len(videos))
processes = []
for index, video in enumerate(videos):
    video_utlity_use_case= VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    process = Process(target=video_utlity_use_case._multiprocessing_wrapper_audio_extraction_wrapper, args=(results,index))
    processes.append(process)
    process.start()
for process in processes:
    process.join()
extract_audio_time_processes = time() -extract_audio_time_processes
print(f"Time taken for processing extraction is {extract_audio_time_processes} seconds")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package stopwords is already 

2024-06-24 11:43:07.808104+00:00 INFO: Extracting audio using process 0 {} () 

2024-06-24 11:43:07.808287+00:00 INFO: Extracting audio from video If Educational Videos Were Filmed Like Music Videos {} () 

MoviePy - Writing audio in tmp_audio/1ab8fb4d-5cf8-4e8c-91b8-e8a30aec0f21.wav
2024-06-24 11:43:07.846199+00:00 INFO: Extracting audio using process 1 {} () 

2024-06-24 11:43:07.846362+00:00 INFO: Extracting audio from video Casually Explained: Guide to College and University {} () 

MoviePy - Writing audio in tmp_audio/5684b307-a3ae-4fd3-a687-678bf3b0df78.wav
2024-06-24 11:43:07.925768+00:00 INFO: Extracting audio using process 2 {} () 

2024-06-24 11:43:07.925938+00:00 INFO: Extracting audio from video Potter Puppet Pals: The Mysterious Ticking Noise {} () 

MoviePy - Writing audio in tmp_audio/247a3024-a5ca-481f-a1dd-a69acab7c50a.wav


chunk:   0%|          | 0/3746 [00:00<?, ?it/s, now=None]/s, now=None]

2024-06-24 11:43:07.995977+00:00 INFO: Extracting audio using process 3 {} () 

2024-06-24 11:43:07.996110+00:00 INFO: Extracting audio from video Fastest Rap EVER {} () 

MoviePy - Writing audio in tmp_audio/3d81722b-8a92-4a88-a6f7-36bc1e9651de.wav
2024-06-24 11:43:08.106898+00:00 INFO: Extracting audio using process 6 {} () 

2024-06-24 11:43:08.107009+00:00 INFO: Extracting audio from video L.O - Don't wanna be a flirt | Dir.Liquidartsmedia {} () 

MoviePy - Writing audio in tmp_audio/87e1b42f-cc02-4047-afb3-cfe71696c23c.wav
2024-06-24 11:43:08.086369+00:00 INFO: Extracting audio using process 11 {} () 

2024-06-24 11:43:08.086621+00:00 INFO: Extracting audio from video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} () 

MoviePy - Writing audio in tmp_audio/be0e260b-7477-468b-90b2-b36be2d7d9f3.wav
2024-06-24 11:43:08.088368+00:00 INFO: Extracting audio using process 8 {} () 

2024-06-24 11:43:08.088495+00:00 INFO: Extracting audio from video Best 3 minu

chunk:  37%|███▋      | 1056/2860 [00:00<00:00, 4517.93it/s, now=None]

...

MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
MoviePy - Done.


chunk:  65%|██████▍   | 2418/3746 [00:00<00:00, 3385.92it/s, now=None]

MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
MoviePy - Done.


MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
MoviePy - Done.


Time taken for processing extraction is 3.456768035888672 seconds


### Comparison of sequential, threading, multiprocessing for extracting audio

In [17]:
pd.DataFrame({
    "Method": ["Sequential", "Threading", "Multiprocessing"],
    "Time Taken": [extract_audio_time_seq, extract_audio_time_threading, extract_audio_time_processes]
})

,Method,Time Taken
0,Sequential,9.156676
1,Threading,10.704738
2,Multiprocessing,3.456768


## Short Report

# CineSense Project Implementation:


## Architecture

I’ve selected the hexagonal (also called ports and adapters) architecture for this project as it is widely used in industry settings due to its scalability and reusability. It has been used by Netflix to manage their flow of large amounts of data from different sources due to its ability to  swap data sources without impacting business logic (Damir Svrtan 2020). 
The architecture breaks the program down into Domain, Services, Use cases and Adapters. The Domain is the broad data that we’re working with – in this case YouTube videos and their content. Services represent the results we want to obtain from the domain – in this case a Jupyter file which presents the log of what we did with the data, and the #1 downloaded videos, #2 transcribed audio, #3 sentiment analysis, #4 emotion extraction and #5 text translation into Spanish – the result of which can be found in the output files. Because the actual implementation – which lives in the adapters folder – is decoupled from the business logic, the application can be easily maintained should the requirements change – e.g. if we want to use a different library to extract emotions, we simply modify the adapter. 

## Use cases

There are two major use cases: the actual downloading of the YouTube videos and the actions we can perform on the content, such as sentiment analysis. I have enveloped the actions into a single use case called utilities to separate the obtaining of the data and the analysis of the data. 
Downloading Videos
Downloading videos only requires one method. I have compared the speed of download for sequential download, threading and multiprocessing. The results of this can be found in the pandas data frame in the Jupyter file. Unsurprisingly, the sequential method is the least efficient. The multiprocessing method is the most efficient, as this is a CPU bound task and multiprocessing is taking advantage of multi-core processors and processes downloads in parallel. The time saving is evident, as multiprocessing is roughly 4x quicker than sequential download in this case. Threading also brings a much improved result compared to sequential download, however, as this is not an I/O bound task, it is less efficient than multiprocessing. 

## Analysing Videos
The video analysis consists of extracting audio, transcribing audio, extracting sentiment analysis and extracting emotions. I have used the MoviePy library for extracting the audio and speechrecognition for transcribing it. For extracting emotions I’ve used the text2emotion library, for sentiment analysis I’ve used the TextBlob library (returning the polarity and sensitivity) and for translation I’ve relied on Google Translate. The results for each video can be found in the respective folder, labelled according to the name of the video. 
As the extract emotions, extract sentiment analysis and translate methods repeatedly rely on the results of the extract audio and transcribe audio methods, I have used caching to store the extracted and transcribed audio, which significantly increased the overall speed of the program. 
I have also compared the sequential, threading and multiprocessing methods for extracting audio. Once again, the multiprocessing method is the most efficient due to its suitability for CPU bound tasks. Surprisingly, the sequential method performed marginally better than the threading method. There is a variety of possible reasons for this, with a likely one being that threading creates a lot of overhead in terms of managing context switching and synchronization. Furthermore, threads working on extracting one audio might lead to contention for resources, which is avoided when using the sequential method. 

## Logging

I have implemented a simple logger to inform the user regarding the state of the processing of individual videos, as well as any potential errors. 
There is also the option to use loguru for a more scaled up logging experience. 


## Works Cited
Damir Svrtan, Sergii Makagon. 2020. Netflix Technology Blog. 10 March. Accessed June 24, 2024. https://netflixtechblog.com/ready-for-changes-with-hexagonal-architecture-b315ec967749.


